In [27]:
import os, sys
module_path = os.path.abspath(os.path.join('fede'))
if module_path not in sys.path:
    sys.path.append(module_path)

from client import Client
from sklearn.utils import shuffle
from global_model import GlobalModel
from supported_modles import Supported_modles
import numpy as np
import utils
from sklearn.metrics import accuracy_score, f1_score

import random

In [28]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [29]:
selected_model = Supported_modles.logistic_regression

In [30]:
#NODEs
clients, test_x, test_y = utils.set_data(False,True)

Creating node1.
Creating node2.
Creating node3.
Creating node4.
Creating node5.


In [31]:
clients[0].x.shape

(20000, 6)

In [32]:
f1_before = []
for client in clients:
    client.init_empty_model(selected_model,0.001)
    client.train_model()
    y_hat = client.model.predict(test_x)
    f1_before.append(f1_score(test_y,y_hat, average="binary"))

In [33]:
print(f1_before)

[0.7676124157344987, 0.7672480835462727, 0.7773463257381493, 0.7689692131426248, 0.7676535899333826]


In [34]:
global_model = GlobalModel(selected_model)
global_model.init_global_model(selected_model,78)

In [38]:
number_of_rounds = 20
for _ in range(number_of_rounds):
    RANDOM_NUMBER_OF_CLIENTS = random.randint(1, 5)

    print(f'Starting new round!')
    f1s = np.empty((0,RANDOM_NUMBER_OF_CLIENTS), float)
    data_weights = arr = np.empty((0,RANDOM_NUMBER_OF_CLIENTS), float)

    applicable_clients = random.sample((clients), RANDOM_NUMBER_OF_CLIENTS)

    for client in applicable_clients:
        print(f'Client name: {client.name}')
        y_hat = client.model.predict(test_x)
        f1_local = f1_score(test_y,y_hat)
        f1s = np.append(f1s, f1_local)
        data_weights = np.append(data_weights, client.x.shape[0])
    
    # print(f'Weights: {f1s/sum(f1s)}')
    # print(f'weigths data: {data_weights/sum(data_weights)}')
    
    round_weights = (f1s/sum(f1s) + data_weights/sum(data_weights))/2
    
    # print(round_weights)

    global_model.update_global_model(applicable_clients,round_weights,selected_model)

    for client in clients:
        y_hat = global_model.model.predict(test_x)
        f1_fede = f1_score(test_y,y_hat)
        y_hat = client.model.predict(test_x)
        f1_local = f1_score(test_y,y_hat)

        if f1_fede > f1_local:
            print("f1 fede was better")
            client.F1 = f1_fede
            # local was better set to local values
            if selected_model == Supported_modles.MLP_classifier:
                client.model.intercepts_ = global_model.model.intercepts_
                client.model.coefs_ = global_model.model.coefs_
            else:
                client.model.intercept_ = global_model.model.intercept_
                client.model.coef_ = global_model.model.coef_
                
    print(f'++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

Starting new round!
Client name: node2
Client name: node1
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Starting new round!
Client name: node1
Client name: node2
Client name: node3
f1 fede was better
f1 fede was better
f1 fede was better
f1 fede was better
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Starting new round!
Client name: node4
Client name: node3
Client name: node1
Client name: node5
Client name: node2
f1 fede was better
f1 fede was better
f1 fede was better
f1 fede was better
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Starting new round!
Client name: node2
Client name: node5
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Starting new round!
Client name: node3
Client name: node2
f1 fede was better
f1 fede was better
f1 fede was better
f1 fede was better
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Starting ne

In [39]:
f1_fedavg = []
for client in clients:
    x = test_x
    y_hat = global_model.model.predict(x)
    f1_fedavg.append(f1_score(test_y,y_hat))
print(f1_fedavg)

[0.7773463257381493, 0.7773463257381493, 0.7773463257381493, 0.7773463257381493, 0.7773463257381493]


In [40]:
i = 0
for client in clients:
    print(f'Difference {f1_fedavg[i]-f1_before[i]}')
    i += 1

Difference 0.009733910003650648
Difference 0.010098242191876605
Difference 0.0
Difference 0.008377112595524538
Difference 0.009692735804766706
